#### Spotify API:
- API connection
- Data sorting
- Data to CSV

In [ ]:
# API connection

import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


CLIENT_ID = 'xx'
CLIENT_SECRET = 'xx'


auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [2]:
# Data sorting

spotify_data = []

def data_extraction_s(list_url):  # pongo la url de la playlist de spotify en la funcion, cada url es una playlist de un año.

    for url in list_url:  # recorre cada url y saca informacion

        playlist = sp.playlist(url, fields=None, market=None, additional_types= 'track') # descarga informacion completa de la playlist por separado 

        
        for item in playlist["tracks"]["items"]:  # itera por cada pist de la playlist y saca track, artist y año
            track = item["track"]
            artist = item["track"]["artists"]
            eurovisionyear = playlist['name']

            spotify_data.append({     # guarda los datos que le pido en un diccionario
                "Artist": artist[0]["name"].title(),
                "Track": track["name"].title(),
                "Spotify_Duration": track["duration_ms"],
                "Spotify_EurovisionYear" : eurovisionyear.title(),
                "Spotify_Popularity": track["popularity"]
                })
        
        

In [3]:
# Data to CSV: convierto la info en un csv

raw_data_s = data_extraction_s([
    "https://open.spotify.com/playlist/3ZdQUt8Tmtt7oOU8UM2koe?si=eea27348cf654d21", 
    "https://open.spotify.com/playlist/5h0sQpJnLVzgy5iOH1UNcl?si=3a1d926528484506", 
    "https://open.spotify.com/playlist/3EBtzzGyrlDUr1QvXVGz0h?si=DMNkLRIjQE6THtHUYc-oQw",
    "https://open.spotify.com/playlist/61fyG82H3Cha91cgKgQT6S?si=o-Xs3DfHRZyRuR8qaY9mxAW",
    "https://open.spotify.com/playlist/2HMW5JNZNh9HLfIHNzY0DP?si=f29ed55294394682"
    ])

df_spotify = pd.DataFrame(spotify_data)
df_spotify

df_spotify.to_csv('Spotify_data.csv')


In [4]:
df_spotify

,Artist,Track,Spotify_Duration,Spotify_EurovisionYear,Spotify_Popularity
0,Chingiz,Truth - Radio Edit,209411,Eurovision 2019,0
1,John Lundvik,Too Late For Love,178133,Eurovision 2019,0
2,Kate Miller-Heidke,Zero Gravity,177443,Eurovision 2019,38
3,Michela,Chameleon,179698,Eurovision 2019,0
4,Mahmood,Soldi,195476,Eurovision 2019,0
...,...,...,...,...,...
205,Fahree,Özünlə Apar,182090,Eurovision 2024,40
206,Sarah Bonnici,Loop,179727,Eurovision 2024,42
207,Megara,11:11,180087,Eurovision 2024,39
208,Natalia Barbu,In The Middle,178853,Eurovision 2024,37


#### Eurovision API:
- Api connection & Sorting
- Data to CSV

In [5]:
import requests
import pandas as pd
import json

In [6]:
# API connection & Sorting

eurovision_data = []
def data_extraction_e(list_years):

    eu_data = requests.get(f"https://eurovisionapi.runasp.net/api/contests/{list_years}/")

    eu_data_json = eu_data.json()
    
    if 'contestants' in eu_data_json:
        
        for idx, i in enumerate(eu_data_json["contestants"], start=1):

            contestant_name = i["artist"]
            song_name = i["song"]
            country = i["country"]

            eurovision_data.append({
                "Ranking": idx,
                "Artist": contestant_name.title(),
                "Track": song_name.title(),
                "Country": country.upper(),
                "Eurovision_Year": list_years
                })

In [7]:
# Data to CSV

eurovision_years= [2019,2021,2022,2023,2024]

for year in eurovision_years:
    raw_data_e = data_extraction_e(year)
df_eurovision = pd.DataFrame(eurovision_data)
df_eurovision

df_eurovision.to_csv('Eurovision_data.csv', index=False)


#### LastFM API:


- Artists

In [8]:
import pandas as pd
import requests
import json
import re  # Importar el módulo para limpiar HTML

In [ ]:
# FUNCION PARA INFO DE UN ARTISTA:

api_key = "xx"
share_secret = "xx"

def obtener_info_artista(artist_name, api_key):  

    url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artist_name}&api_key={api_key}&format=json"
    
    response = requests.get(url)  
    info_artist = response.json() 

  
    if "error" in info_artist:
        print(f"Error en la API: {info_artist['message']}")
        return pd.DataFrame()  

    
    nombre = info_artist["artist"].get("name")
    url = info_artist["artist"].get("url") 
    oyentes = info_artist["artist"]["stats"].get("listeners")
    reproducciones = info_artist["artist"]["stats"].get("playcount")

    bio_summary = info_artist["artist"]["bio"].get("summary")
    bio_summary_clean = re.sub(r"<.*?>", "", bio_summary)  # Eliminar etiquetas HTML

    generos = [tag["name"] for tag in info_artist["artist"]["tags"]["tag"][:3]] 
    generos_texto = ", ".join(generos)  # utilizo este codigo para que aparezca como cadena de texto y no como lista con corchetes y comillas.

    artistas_similares = [artista["name"] for artista in info_artist["artist"]["similar"]["artist"][:3]]  
    artistas_similares_texto = ", ".join(artistas_similares)   # igual que en genero lo convierto en cadena de texto.

    return {                    
        "artista": nombre,
        "url":url,
        "oyentes": oyentes,
        "reproducciones": reproducciones,
        "generos": generos_texto,
        "artistas_similares": artistas_similares_texto,
        "biografia_resumen": bio_summary_clean
    }



In [10]:
#lista artistas por año:

popular_artists_2019 = ["Duncan Laurence", "KEiiNO","MARUV", "Tamta","Kate Miller-Heidke" ]
popular_artists_2021 = ["Barbara Pravi", "Måneskin","Efendi", "Tusse", "THE ROOP"]
popular_artists_2022 = ["Mahmood", "LUM!X", "S10", "MARO", "Subwoolfer"]
popular_artists_2023 = ["Loreen", "Alessandra","Käärijä", "Marco Mengoni", "BLANKA"]
popular_artists_2024 = ["Joost", "Nemo", "Slimane", "Angelina Mango", "Baby Lasagna"]

artists_per_years = [
    popular_artists_2019,
    popular_artists_2021,
    popular_artists_2022,
    popular_artists_2023,
    popular_artists_2024]

years =[2019, 2021, 2022, 2023, 2024]

def get_info_artists_per_year(artists_per_years, api_key, years):
    data_artists = []  
    
    for artists, year in zip(artists_per_years, years): # ZIP permite combinar dos o mas listas de manera que puedas iterar sobre ellas al mismo tiempo.
        for artist in artists:
            info = obtener_info_artista(artist, api_key)  
            if info: 
                info["year"] = year 
                data_artists.append(info)  
    
    
    df_artists = pd.DataFrame(data_artists)
    return df_artists

In [11]:
df_artists = get_info_artists_per_year(artists_per_years, api_key, years)
print(df_artists.head())

              artista                                           url oyentes  \
0     Duncan Laurence     https://www.last.fm/music/Duncan+Laurence  576696   
1              Keiino              https://www.last.fm/music/Keiino   65330   
2               MARUV               https://www.last.fm/music/MARUV   85687   
3               Tamta               https://www.last.fm/music/Tamta   49815   
4  Kate Miller-Heidke  https://www.last.fm/music/Kate+Miller-Heidke  137329   

  reproducciones                                          generos  \
0        8434058                            pop, Eurovision, soul   
1        2096077                       pop, norwegian, Eurovision   
2        1955171                         pop, electronic, Ukraine   
3         917272                            Greek, pop, Greek Pop   
4        2225767  australian, singer-songwriter, female vocalists   

                              artistas_similares  \
0       Cornelia Jakobs, Rosa Linn, Gjon's Tears   
1     

In [12]:
df_artists.to_csv('LastFM_25_Artists_Info.csv', index=False)
print(df_artists)

               artista                                               url  \
0      Duncan Laurence         https://www.last.fm/music/Duncan+Laurence   
1               Keiino                  https://www.last.fm/music/Keiino   
2                MARUV                   https://www.last.fm/music/MARUV   
3                Tamta                   https://www.last.fm/music/Tamta   
4   Kate Miller-Heidke      https://www.last.fm/music/Kate+Miller-Heidke   
5        Barbara Pravi           https://www.last.fm/music/Barbara+Pravi   
6             Måneskin           https://www.last.fm/music/M%C3%A5neskin   
7               efendi                  https://www.last.fm/music/efendi   
8                Tusse                   https://www.last.fm/music/Tusse   
9             THE ROOP                https://www.last.fm/music/THE+ROOP   
10             Mahmood                 https://www.last.fm/music/Mahmood   
11               LUM!X                 https://www.last.fm/music/LUM%21X   
12          

In [13]:
from IPython.display import display

display(df_artists)

,artista,url,oyentes,reproducciones,generos,artistas_similares,biografia_resumen,year
0,Duncan Laurence,https://www.last.fm/music/Duncan+Laurence,576696,8434058,"pop, Eurovision, soul","Cornelia Jakobs, Rosa Linn, Gjon's Tears",Duncan Laurence is a Dutch singer who represen...,2019
1,Keiino,https://www.last.fm/music/Keiino,65330,2096077,"pop, norwegian, Eurovision",,Keiino (stylised as KEiiNO) are a Norwegian el...,2019
2,MARUV,https://www.last.fm/music/MARUV,85687,1955171,"pop, electronic, Ukraine","LOBODA, SHLAKOBLOCHINA, Елена Темникова","Anna Korsun (Ukrainian: Aнна Корсун, born Febr...",2019
3,Tamta,https://www.last.fm/music/Tamta,49815,917272,"Greek, pop, Greek Pop","Eleni Foureira, Marina Satti, Elena Tsagrinou","Tamta Goduadze (Greek: Τάμτα Γκοντουάτζε, Geor...",2019
4,Kate Miller-Heidke,https://www.last.fm/music/Kate+Miller-Heidke,137329,2225767,"australian, singer-songwriter, female vocalists","Lisa Mitchell, Meg Washington, Missy Higgins",Kate Miller-Heidke (born 16 November 1981) is ...,2019
5,Barbara Pravi,https://www.last.fm/music/Barbara+Pravi,162055,2607439,"chanson, pop, french","La Zarra, Pomme, Zaho de Sagazan",Barbara Pravi (real name: Barbara Piévic) is a...,2021
6,Måneskin,https://www.last.fm/music/M%C3%A5neskin,1494630,79614635,"rock, glam rock, italian","Palaye Royale, Mahmood, The Pretty Reckless","Måneskin (Italian pronunciation: ˈmɔːneskin, D...",2021
7,efendi,https://www.last.fm/music/efendi,60973,981258,"pop, azerbaijan, Eurovision",,Samira Efendi is an Azerbaijani singer. She wa...,2021
8,Tusse,https://www.last.fm/music/Tusse,53196,539433,"swedish, soul, melodifestivalen 2021","Natalia Gordienko, Dotter, Robin Bengtsson","Tousin Michael Chiza (born 1 January 2002), kn...",2021
9,THE ROOP,https://www.last.fm/music/THE+ROOP,72307,1208434,"pop, pop rock, Eurovision","Gjon's Tears, Efendi, Natalia Gordienko",The Roop are a Lithuanian pop rock band from V...,2021


In [ ]:

def get_top_tracks(artist, year): #year parameter included for future artist/year association in DataFrame o CSV.
  
    url = f"http://ws.audioscrobbler.com/2.0/?method=artist.gettoptracks&artist={artist}&api_key={MY_API_KEY}&format=json"
    response = requests.get(url)

    if response.status_code == 200:
        data_tracks = response.json()
        if "toptracks" in data_tracks and "track" in data_tracks["toptracks"]:
            return [{"Artist": artist, "Year": year, "Track": track["name"]} for track in data_tracks["toptracks"]["track"][:3]]
        else:
            print(f"❌ Error fetching data for {artist} ({year}): {response.status_code}")



In [ ]:
# Store all data in a single list
all_tracks = []

# Iterate through years and artists
for year, artists in top_artists_by_year.items():
    for artist in artists:
        all_tracks.extend(get_top_tracks(artist, year))

# Convert the list to a DataFrame
df = pd.DataFrame(all_tracks)

# Save to a single CSV file
df.to_csv("all_top_tracks.csv", index=False)

print("📁 Data successfully saved to 'all_top_tracks.csv'")


#### MySQL Data Base
- MySQL connection
- Schema Creation and Use
- Tables Creation

In [ ]:
# MySQL connection

import mysql.connector

try:
    cnx = mysql.connector.connect(user='root', password='',
                              host='127.0.0.1')
    

    mycursor = cnx.cursor()
    

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

In [15]:
# Schema Creation and Use

query_create_schema = """CREATE SCHEMA StreamMusic"""
mycursor.execute(query_create_schema)
mycursor.execute('USE StreamMusic')

In [16]:
# Tables creation


the_tables = {  'API_Eurovision' :          """CREATE TABLE api_eurovision (
                                                Ranking INT NOT NULL,
                                                Artist VARCHAR(45) NOT NULL,                              
                                                Track VARCHAR(70) NOT NULL,
                                                Country VARCHAR(3) NOT NULL,
                                                Eurovision_Year YEAR NOT NULL)""",
                                                
                'API_Spotify_Eurovision' :    """CREATE TABLE api_spotify_merged_eurovision (
                                                Artist VARCHAR(45),
                                                Track VARCHAR(70),
                                                Spotify_Duration INT,
                                                Spotify_EurovisionYear VARCHAR(25),
                                                Spotify_Popularity VARCHAR(10),
                                                Eurovision_Ranking INT,
                                                Eurovision_Country CHAR(2),
                                                PRIMARY KEY (Artist, Track)
                                                )""", 
                
                'API_LastFM_Info_Artists' :  """CREATE TABLE api_lastfm_info_artists (
                                                Artist VARCHAR(45),
                                                Listeners INT,
                                                Playcount INT,
                                                Genre1 VARCHAR(45),
                                                Genre2 VARCHAR(45),
                                                Genre3 VARCHAR(45),
                                                Similar_Artists1 VARCHAR(45),
                                                Similar_Artists2 VARCHAR(45),
                                                Similar_Artists3 VARCHAR(45),
                                                PRIMARY KEY (Artist)
                                                )""",
                
                'API_LastFM_Top_Tracks' :    """CREATE TABLE api_lastfm_top_tracks (
                                                Artist VARCHAR(70) NOT NULL,
                                                Release_year YEAR NOT NULL,
                                                Top_track VARCHAR(120)
                                                )""" 
                }



for k, v in the_tables.items():
        print(f'Table created: {k}')
        mycursor.execute(v)

        print('------------')



Table created: API_Eurovision
------------
Table created: API_Spotify_Eurovision
------------
Table created: API_LastFM_Info_Artists
------------
Table created: API_LastFM_Top_Tracks
------------


#### Data Import

- Method
- Queries
- Data Import

In [17]:
# Data Import Method

def data_import(query,values):

    mycursor = cnx.cursor()
    mycursor.executemany(query, values)
    print(mycursor.rowcount,'values entered')
    cnx.commit()
 

In [18]:
# Data Import Queries 
query_e_import =            """ INSERT INTO API_EUROVISION (Ranking, Artist, Track, Country, Eurovision_Year)
                                VALUES (%s, %s, %s, %s, %s) """

query_s_e_import =            """ INSERT INTO api_spotify_merged_eurovision (Artist, Track, Spotify_Duration, Spotify_EurovisionYear, Spotify_Popularity)
                                VALUES (%s, %s, %s, %s, %s) """

query_l_artist_import =     """ INSERT INTO api_lastfm_info_artists (Artist, Listeners, Playcount, Genre1, Genre2, Genre3, Similar_Artists1, Similar_Artists2, Similar_Artists3)
                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) """

query_l_toptracks_import =  """ INSERT INTO api_lastfm_top_tracks (Artist, Release_year, Top_track)
                                VALUES (%s, %s, %s) """



In [19]:
# Eurovision Data Import

e_file = pd.read_csv('clean_csv/Eurovision_data_clean.csv', sep=";")
e_values = list(e_file.itertuples(index=False, name=None))
eurovisionimp = data_import(query_e_import, e_values)

194 values entered


In [20]:
# Spotify Data Import

s_file = pd.read_csv('clean_csv/Spotify_data_clean.csv', sep=";")
s_file = s_file.iloc[:, 1:]
s_values = list(s_file.itertuples(index = False, name=None))
spotifyimp = data_import(query_s_e_import, s_values) 

195 values entered


In [21]:
#LastFM Artist Import

l_a_file = pd.read_csv('clean_csv/info_artist_clean.csv', sep = ';')

l_a_file = l_a_file.fillna('')

l_a_values = list(l_a_file.itertuples(index=False, name=None))
lastfmtracksimp = data_import(query_l_artist_import, l_a_values )


25 values entered


In [22]:
#LastFM Tracks Import

l_t_file = pd.read_csv('clean_csv/all_top_tracks.csv')

l_t_values = list(l_t_file.itertuples(index=False, name=None))
lastfmtracksimp = data_import(query_l_toptracks_import, l_t_values )

75 values entered
